In [1]:
import os
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import pickle
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
start = time.time()
base_dir = r"E:\Tennis Schema\data\raw"
cache_file = "cached_tables.pkl"
pd.options.io.parquet.engine = "pyarrow"

In [3]:
if os.path.exists(cache_file):
    with open(cache_file, "rb") as f:
        dfs = pickle.load(f)
    print(" Data loaded from cache.")
else:
    dfs = {}

    def read_parquet_group(folder, prefix, table_name):
        """Reads all parquet files in a folder that start with a specific prefix (parallel + safe)."""
        folder_path = os.path.join(base_dir, folder)
        if not os.path.exists(folder_path):
            print(f" Folder not found: {folder_path}")
            return

        parquet_files = [
            os.path.join(folder_path, f)
            for f in os.listdir(folder_path)
            if f.startswith(prefix) and f.endswith(".parquet")
        ]

        if not parquet_files:
            print(f" No parquet files found for {table_name}")
            return

        def load_one(path):
            try:
                return pd.read_parquet(path)
            except Exception as e:
                print(f" Error reading {path}: {e}")
                return pd.DataFrame()

        max_workers = min(32, os.cpu_count() * 2)
        with ThreadPoolExecutor(max_workers=max_workers) as ex:
            dfs_list = list(ex.map(load_one, parquet_files))

        non_empty = [d for d in dfs_list if not d.empty]
        if non_empty:
            dfs[table_name] = pd.concat(non_empty, ignore_index=True)
            print(f" Loaded {table_name:25} → {dfs[table_name].shape}")
        else:
            print(f" No data in {table_name}")

    match_folder = "raw_match_parquet"
    read_parquet_group(match_folder, "event_", "MatchEventInfo")
    read_parquet_group(match_folder, "home_team_", "MatchHomeTeamInfo")
    read_parquet_group(match_folder, "away_team_", "MatchAwayTeamInfo")
    read_parquet_group(match_folder, "home_team_score_", "MatchHomeScoreInfo")
    read_parquet_group(match_folder, "away_team_score_", "MatchAwayScoreInfo")
    read_parquet_group(match_folder, "tournament_", "MatchTournamentInfo")
    read_parquet_group(match_folder, "season_", "MatchSeasonInfo")
    read_parquet_group(match_folder, "round_", "MatchRoundInfo")
    read_parquet_group(match_folder, "venue_", "MatchVenueInfo")
    read_parquet_group(match_folder, "time_", "MatchTimeInfo")

    read_parquet_group("raw_odds_parquet", "odds_", "OddsInfo")
    read_parquet_group("raw_point_by_point_parquet", "pbp_", "GameInfo")
    read_parquet_group("raw_statistics_parquet", "statistics_", "StatisticInfo")
    read_parquet_group("raw_tennis_power_parquet", "power_", "PowerInfo")
    read_parquet_group("raw_votes_parquet", "votes_", "MatchVotesInfo")

    with open(cache_file, "wb") as f:
        pickle.dump(dfs, f)

print("\n Summary of loaded tables:")
for name, df in dfs.items():
    print(f"{name:25} → {df.shape}")

print("\n All tables ready for analysis!")
end = time.time()
print("Execution time = ", end - start)

 Data loaded from cache.

 Summary of loaded tables:
MatchEventInfo            → (16873, 10)
MatchHomeTeamInfo         → (29262, 31)
MatchAwayTeamInfo         → (28563, 31)
MatchHomeScoreInfo        → (16873, 14)
MatchAwayScoreInfo        → (16873, 14)
MatchTournamentInfo       → (16873, 16)
MatchSeasonInfo           → (16873, 4)
MatchRoundInfo            → (9243, 5)
MatchVenueInfo            → (16749, 5)
MatchTimeInfo             → (16873, 7)
OddsInfo                  → (28790, 11)
GameInfo                  → (1254739, 13)
StatisticInfo             → (665589, 13)
PowerInfo                 → (230581, 5)
MatchVotesInfo            → (16873, 3)

 All tables ready for analysis!
Execution time =  1.5201992988586426


Q1:How many tennis player are included in the data set?

In [23]:
df_MatchHomeTeam=dfs['MatchHomeTeamInfo']
df_MatchAwayTeam=dfs['MatchAwayTeamInfo']

df_MatchAwayTeam['player_id']=df_MatchAwayTeam['player_id'].astype(str).str.strip()
df_MatchHomeTeam['player_id']=df_MatchHomeTeam['player_id'].astype(str).str.strip()
df_MatchAwayTeam = df_MatchAwayTeam[df_MatchAwayTeam['player_id'].notna()]
df_MatchHomeTeam = df_MatchHomeTeam[df_MatchHomeTeam['player_id'].notna()]

players = pd.concat([df_MatchAwayTeam['player_id'],
                     df_MatchHomeTeam['player_id']]).drop_duplicates()

len(players)



2645